Bottleneck features

In [ ]:
from keras import applications

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'D:\Docs\Python_code\DogHotDogNotDog\Data\Train_data'
validation_data_dir = 'D:\Docs\Python_code\DogHotDogNotDog\Data\Val_data'
nb_train_samples = train_samples #2000
nb_validation_samples = val_samples #800
epochs = 50

def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
            train_data_dir,
            target_size=(img_width, img_height),
            batch_size=batch_size,
            class_mode=None,
            shuffle=False)
    
    train_labels = generator.classes
#     generator = datagen.flow(train_Images, dummy_y_train, batch_size=batch_size, 
#                                       shuffle=False, sample_weight=None, seed=None, 
#                                       save_to_dir=None, save_prefix='', save_format='jpg', subset=None)
    
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
#     generator = datagen.flow(val_Images, dummy_y_val, batch_size=batch_size, 
#                                       shuffle=False, sample_weight=None, seed=None, 
#                                       save_to_dir=None, save_prefix='', save_format='jpg', subset=None)
    
    val_labels = generator.classes
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)
    
    return train_labels, val_labels

In [ ]:
def train_top_model(train_labels, val_labels):
    train_data = np.load(open('bottleneck_features_train.npy','rb'))
    train_labels = train_labels[:len(train_data)]
    #train_labels = dummy_y_train[:len(train_data)] #np.array(
        #[0] * (nb_train_samples / 2) + [1] * (nb_train_samples / 2))

    validation_data = np.load(open('bottleneck_features_validation.npy','rb'))
    validation_labels = val_labels[:len(validation_data)] #dummy_y_val[:len(validation_data)] #np.array(
        #[0] * (nb_validation_samples / 2) + [1] * (nb_validation_samples / 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

In [ ]:
train_labels, val_labels = save_bottlebeck_features()

In [ ]:
train_top_model(train_labels, val_labels)

In [ ]:
train_data = np.load(open('bottleneck_features_train.npy','rb'))
train_data.shape